In [ ]:
import pandas as pd
import os

In [ ]:
len(os.listdir('/content/drive/MyDrive/123뉴스분류'))

123

In [ ]:
# 1번 파인튜닝

In [ ]:
company = ['카카오뱅크', '한미반도체', 'NAVER', '대웅제약', '삼성화재', '포스코인터내셔널', '한화', '신한지주', 'GS', '한국콜마']

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification


for com in company:
# ==========================================
# 1. 설정 (학습 때와 동일하게 맞춰주세요)
# ==========================================
  MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real" # 저장된 경로
  INPUT_CSV = f"/content/drive/MyDrive/news_clean/{com}_clean.csv"  # 예측하고 싶은 파일
  OUTPUT_CSV = f"/content/drive/MyDrive/1번파인튜닝결과/{com}_predicted_results.csv" # 결과 저장 경로

  MAX_LEN = 256
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # ==========================================
  # 2. 모델 및 토크나이저 로드
  # ==========================================
  print("모델 로딩 중...")
  tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
  model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
  model.eval()

  # ==========================================
  # 3. 데이터 로드 및 전처리
  # ==========================================
  df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

  # 제목과 본문 합치기 (학습 때와 동일한 포맷)
  # 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
  if "본문" not in df.columns:
      df["본문"] = ""

  texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

  # ==========================================
  # 4. 예측 (Inference)
  # ==========================================
  results = []
  inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

  print(f"{len(texts)}개의 기사 분석 시작...")

  with torch.no_grad():
      for text in tqdm(texts):
          # 토큰화
          inputs = tokenizer(
              text,
              return_tensors="pt",
              truncation=True,
              max_length=MAX_LEN,
              padding="max_length"
          ).to(device)

          # 모델 추론
          outputs = model(**inputs)
          logits = outputs.logits

          # 확률값 계산 (Softmax)
          probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

          # 가장 높은 확률의 인덱스 선택
          pred_id = np.argmax(probs)
          pred_label = inv_label_map[pred_id]

          # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
          results.append({
              "pred_label": pred_label,
              "prob_neg": round(probs[0], 4),
              "prob_neu": round(probs[1], 4),
              "prob_pos": round(probs[2], 4)
          })

  # ==========================================
  # 5. 결과 합치기 및 저장
  # ==========================================
  pred_df = pd.DataFrame(results)
  final_df = pd.concat([df, pred_df], axis=1)

  # 감성을 한글로 보고 싶다면 추가
  final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

  final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
  print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

모델 로딩 중...


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


10229개의 기사 분석 시작...


100%|██████████| 10229/10229 [03:58<00:00, 42.93it/s]
The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/1번파인튜닝결과/카카오뱅크_predicted_results.csv
모델 로딩 중...
3278개의 기사 분석 시작...


...

✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/1번파인튜닝결과/신한지주_predicted_results.csv
모델 로딩 중...
2096개의 기사 분석 시작...


100%|██████████| 2096/2096 [00:49<00:00, 42.50it/s]
The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/1번파인튜닝결과/GS_predicted_results.csv
모델 로딩 중...
4249개의 기사 분석 시작...


100%|██████████| 4249/4249 [01:39<00:00, 42.71it/s]

✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/1번파인튜닝결과/한국콜마_predicted_results.csv


In [ ]:
# 1번 추가 파인튜닝

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification


for com in company:
# ==========================================
# 1. 설정 (학습 때와 동일하게 맞춰주세요)
# ==========================================
  MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus" # 저장된 경로
  INPUT_CSV = f"/content/drive/MyDrive/news_clean/{com}_clean.csv"  # 예측하고 싶은 파일
  OUTPUT_CSV = f"/content/drive/MyDrive/추가1번파인튜닝결과/{com}_predicted_results_plus.csv" # 결과 저장 경로

  MAX_LEN = 256
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # ==========================================
  # 2. 모델 및 토크나이저 로드
  # ==========================================
  print("모델 로딩 중...")
  tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
  model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
  model.eval()

  # ==========================================
  # 3. 데이터 로드 및 전처리
  # ==========================================
  df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

  # 제목과 본문 합치기 (학습 때와 동일한 포맷)
  # 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
  if "본문" not in df.columns:
      df["본문"] = ""

  texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

  # ==========================================
  # 4. 예측 (Inference)
  # ==========================================
  results = []
  inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

  print(f"{len(texts)}개의 기사 분석 시작...")

  with torch.no_grad():
      for text in tqdm(texts):
          # 토큰화
          inputs = tokenizer(
              text,
              return_tensors="pt",
              truncation=True,
              max_length=MAX_LEN,
              padding="max_length"
          ).to(device)

          # 모델 추론
          outputs = model(**inputs)
          logits = outputs.logits

          # 확률값 계산 (Softmax)
          probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

          # 가장 높은 확률의 인덱스 선택
          pred_id = np.argmax(probs)
          pred_label = inv_label_map[pred_id]

          # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
          results.append({
              "pred_label": pred_label,
              "prob_neg": round(probs[0], 4),
              "prob_neu": round(probs[1], 4),
              "prob_pos": round(probs[2], 4)
          })

  # ==========================================
  # 5. 결과 합치기 및 저장
  # ==========================================
  pred_df = pd.DataFrame(results)
  final_df = pd.concat([df, pred_df], axis=1)

  # 감성을 한글로 보고 싶다면 추가
  final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

  final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
  print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

모델 로딩 중...


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


10229개의 기사 분석 시작...


100%|██████████| 10229/10229 [03:59<00:00, 42.65it/s]
The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/추가1번파인튜닝결과/카카오뱅크_predicted_results_plus.csv
모델 로딩 중...
3278개의 기사 분석 시작...


...

✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/추가1번파인튜닝결과/신한지주_predicted_results_plus.csv
모델 로딩 중...
2096개의 기사 분석 시작...


100%|██████████| 2096/2096 [00:49<00:00, 42.58it/s]
The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/추가1번파인튜닝결과/GS_predicted_results_plus.csv
모델 로딩 중...
4249개의 기사 분석 시작...


100%|██████████| 4249/4249 [01:40<00:00, 42.47it/s]

✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/추가1번파인튜닝결과/한국콜마_predicted_results_plus.csv


In [ ]:
# 2번 파인튜닝

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification


for company in range()
  # ==========================================
  # 1. 설정 (학습 때와 동일하게 맞춰주세요)
  # ==========================================
  MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_hope" # 저장된 경로
  INPUT_CSV = "/content/drive/MyDrive/news_clean/KB금융_clean.csv"  # 예측하고 싶은 파일
  OUTPUT_CSV = "/content/drive/MyDrive/뉴스분류/KB금융_predicted_results_hope.csv" # 결과 저장 경로

  MAX_LEN = 256
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # ==========================================
  # 2. 모델 및 토크나이저 로드
  # ==========================================
  print("모델 로딩 중...")
  tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
  model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
  model.eval()

  # ==========================================
  # 3. 데이터 로드 및 전처리
  # ==========================================
  df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

  # 제목과 본문 합치기 (학습 때와 동일한 포맷)
  # 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
  if "본문" not in df.columns:
      df["본문"] = ""

  texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

  # ==========================================
  # 4. 예측 (Inference)
  # ==========================================
  results = []
  inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

  print(f"{len(texts)}개의 기사 분석 시작...")

  with torch.no_grad():
      for text in tqdm(texts):
          # 토큰화
          inputs = tokenizer(
              text,
              return_tensors="pt",
              truncation=True,
              max_length=MAX_LEN,
              padding="max_length"
          ).to(device)

          # 모델 추론
          outputs = model(**inputs)
          logits = outputs.logits

          # 확률값 계산 (Softmax)
          probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

          # 가장 높은 확률의 인덱스 선택
          pred_id = np.argmax(probs)
          pred_label = inv_label_map[pred_id]

          # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
          results.append({
              "pred_label": pred_label,
              "prob_neg": round(probs[0], 4),
              "prob_neu": round(probs[1], 4),
              "prob_pos": round(probs[2], 4)
          })

  # ==========================================
  # 5. 결과 합치기 및 저장
  # ==========================================
  pred_df = pd.DataFrame(results)
  final_df = pd.concat([df, pred_df], axis=1)

  # 감성을 한글로 보고 싶다면 추가
  final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

  final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
  print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

In [ ]:
import pandas as pd

In [ ]:
df_real = pd.read_csv('/content/drive/MyDrive/1번파인튜닝결과/한국콜마_predicted_results.csv')
df_real['sentiment'].value_counts()

,count
sentiment,
중립,2706
호재(긍정),1246
악재(부정),297


In [ ]:
pd.read_csv('/content/drive/MyDrive/1번파인튜닝결과/한국콜마_predicted_results.csv')

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150102,"한국콜마, 기술연구원장에 강학희 전 아모레 상임고문 영입",한국콜마가 강학희 전 아모레퍼시픽 상임고문(사진)을 기술연구원장으로 영입했다. \n...,161890,0,0.0107,0.9881,0.0012,중립
1,20150102,"한국콜마, 아모레 출신 강학희 기술연구원장 영입",한국콜마는 기술연구원장으로 강학희 전 아모레퍼시픽 상임고문(사진)을 영입했\n다고 ...,161890,0,0.0025,0.9962,0.0012,중립
2,20150105,한국콜마 올 경영방침 시이리,한국콜마는 지난 2일 2015년 경영방침을 신뢰와 도덕성을 지키며 지속적인 혁신을 ...,161890,0,0.0028,0.9961,0.0011,중립
3,20150105,"한국콜마, 조직개편 단행 해외사업본부 신설",한국콜마는 지난 2일 세종시 본사에서 시무식을 열고 올해 경영방침을 신뢰와 \n도덕...,161890,0,0.0033,0.9958,0.0010,중립
4,20150105,"[신년사] 윤동한 한국콜마 회장 ""글로벌 도약 원년, 기술경쟁력 강화""",<사진=한국콜마 윤동한 회장이 신년사를 통해 2015년 경영방침 및 글로벌 경영에 ...,161890,0,0.0035,0.9954,0.0010,중립
...,...,...,...,...,...,...,...,...,...
4244,20241226,"한국콜마, 미메틱스와 문어 빨판 모사 기술 활용 화장품 개발",한국콜마가 동 식물 생체모사 스타트업인 미메틱스(MIMETICS)와 손잡고 기능성 ...,161890,0,0.0033,0.9952,0.0016,중립
4245,20241226,"“문어 빨판처럼 피부에 착”...한국콜마, 스타트업과 손잡고 흡수 잘 되는 화장품 개발",생체모사 스타트업 미메틱스와 업무협약 \n문어 빨판구조 패치에 나노전달체 기술 더해...,161890,0,0.0034,0.9939,0.0027,중립
4246,20241226,"한국콜마, ‘문어 빨판’ 모사 기술 개발 나서 화장품 흡수력↑",지난 19일 한국콜마와 동 식물 생체모사 스타트업 미메틱스 관계자들이 ‘생체모사 음...,161890,0,0.0026,0.9959,0.0015,중립
4247,20241226,"한국콜마, '문어 빨판' 모사 기술로 화장품 흡수력 극대화",[머니투데이 조한송 기자] 한국콜마가 동 식물 생체모사 스타트업인 미메틱스(MIME...,161890,0,0.0037,0.9948,0.0015,중립


In [ ]:
df_real

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150102,"한국콜마, 기술연구원장에 강학희 전 아모레 상임고문 영입",한국콜마가 강학희 전 아모레퍼시픽 상임고문(사진)을 기술연구원장으로 영입했다. \n...,161890,0,0.0107,0.9881,0.0012,중립
1,20150102,"한국콜마, 아모레 출신 강학희 기술연구원장 영입",한국콜마는 기술연구원장으로 강학희 전 아모레퍼시픽 상임고문(사진)을 영입했\n다고 ...,161890,0,0.0025,0.9962,0.0012,중립
2,20150105,한국콜마 올 경영방침 시이리,한국콜마는 지난 2일 2015년 경영방침을 신뢰와 도덕성을 지키며 지속적인 혁신을 ...,161890,0,0.0028,0.9961,0.0011,중립
3,20150105,"한국콜마, 조직개편 단행 해외사업본부 신설",한국콜마는 지난 2일 세종시 본사에서 시무식을 열고 올해 경영방침을 신뢰와 \n도덕...,161890,0,0.0033,0.9958,0.0010,중립
4,20150105,"[신년사] 윤동한 한국콜마 회장 ""글로벌 도약 원년, 기술경쟁력 강화""",<사진=한국콜마 윤동한 회장이 신년사를 통해 2015년 경영방침 및 글로벌 경영에 ...,161890,0,0.0035,0.9954,0.0010,중립
...,...,...,...,...,...,...,...,...,...
4244,20241226,"한국콜마, 미메틱스와 문어 빨판 모사 기술 활용 화장품 개발",한국콜마가 동 식물 생체모사 스타트업인 미메틱스(MIMETICS)와 손잡고 기능성 ...,161890,0,0.0033,0.9952,0.0016,중립
4245,20241226,"“문어 빨판처럼 피부에 착”...한국콜마, 스타트업과 손잡고 흡수 잘 되는 화장품 개발",생체모사 스타트업 미메틱스와 업무협약 \n문어 빨판구조 패치에 나노전달체 기술 더해...,161890,0,0.0034,0.9939,0.0027,중립
4246,20241226,"한국콜마, ‘문어 빨판’ 모사 기술 개발 나서 화장품 흡수력↑",지난 19일 한국콜마와 동 식물 생체모사 스타트업 미메틱스 관계자들이 ‘생체모사 음...,161890,0,0.0026,0.9959,0.0015,중립
4247,20241226,"한국콜마, '문어 빨판' 모사 기술로 화장품 흡수력 극대화",[머니투데이 조한송 기자] 한국콜마가 동 식물 생체모사 스타트업인 미메틱스(MIME...,161890,0,0.0037,0.9948,0.0015,중립


In [ ]:
df_real = pd.read_csv('/content/drive/MyDrive/1번파인튜닝결과/한국콜마_predicted_results.csv')
df_real['sentiment'].value_counts()

,count
sentiment,
중립,2706
호재(긍정),1246
악재(부정),297


In [ ]:
df_plus = pd.read_csv('/content/drive/MyDrive/추가1번파인튜닝결과/한국콜마_predicted_results_plus.csv')

In [ ]:
df_plus['sentiment'].value_counts()

,count
sentiment,
호재(긍정),2954
악재(부정),1295


In [ ]:
df_hope = pd.read_csv('/content/drive/MyDrive/뉴스분류/한국콜마_predicted_results_hope.csv')
df_hope['sentiment'].value_counts()

,count
sentiment,
중립,2347
호재(긍정),1359
악재(부정),543


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/news_labeling_100종목/한국콜마_clean_labeling.csv')
df['label'].value_counts()

,count
label,
0,2742
1,1235
-1,272


In [ ]:
idx_list = []
for idx, sentiment in enumerate(df['label']):
  if sentiment != df_hope['pred_label'][idx]:
    idx_list.append(idx)

In [ ]:
df_real.iloc[idx_list, ]

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
12,20150212,"한국콜마, 작년 4Q 이어 올해도 실적 양호 목표가↑-하이",[이데일리 김도년 기자] 하이투자증권은 12일 한국콜마(161890)에 대해 지난해...,161890,0,0.0075,0.9065,0.0860,중립
29,20150330,"한국콜마, 中시장 공략 강화 올 매출 10% 늘것",◆ 기업분석 / 화장품 ODM업체 한국콜마 ◆ \n 한국콜마는 화장품 주문자상표부착...,161890,0,0.0037,0.9919,0.0044,중립
40,20150514,"한국콜마, 화장품 사업 호조 지속될 전망 '목표가↑'-하이",[이데일리 박기주 기자] 하이투자증권이 14일 한국콜마(161890)에 대해 화장품...,161890,0,0.0040,0.9896,0.0063,중립
44,20150514,"""한국콜마, 수출 확대로 할인 요인 해소 목표가↑""-KTB",[ 박희진 기자 ] KTB투자증권은 14일 한국콜마에 대해 올 1분기 실적에서 수출...,161890,0,0.0039,0.9905,0.0055,중립
62,20150520,"한국콜마, 베이징법인 성장 구체화 '목표가↑'-교보",[이데일리 박기주 기자] 교보증권이 20일 한국콜마(161890)에 대해 베이징법인...,161890,0,0.0043,0.9897,0.0060,중립
...,...,...,...,...,...,...,...,...,...
4128,20241106,"한국콜마, 화장품 출시 6개월까지 줄이는 서비스 구축",[대한경제=오진주 기자]한국콜마가 화장품 출시 기간을 줄이는 서비스를 구축했다. \...,161890,0,0.0022,0.9963,0.0015,중립
4138,20241106,한국콜마 “화장품 개발 6개월 단축”,한국콜마가 “화장품 신제품 출시기간을 최대 6개월 단축하는 PPS를 본격 가동한다”...,161890,0,0.0026,0.9959,0.0015,중립
4167,20241111,코스맥스 한국콜마 실적 랠리 'K뷰티 피크아웃' 우려 불식,화장품 제조업자개발생산(ODM) ‘투톱’인 코스맥스와 한국콜마가 올해 3분기 사상 ...,161890,0,0.0069,0.9769,0.0162,중립
4172,20241112,"[단독] 한국콜마, '기술유출 사건' 6년 만 종지부..伊 인터코스 유죄 확정",서울의 한 대형마트에 화장품이 진열돼 있다. 뉴스1 \n[파이낸셜뉴스] 한국콜마의 ...,161890,0,0.0532,0.9331,0.0136,중립


In [ ]:
df_hope.iloc[idx_list, ]

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
12,20150212,"한국콜마, 작년 4Q 이어 올해도 실적 양호 목표가↑-하이",[이데일리 김도년 기자] 하이투자증권은 12일 한국콜마(161890)에 대해 지난해...,161890,1,0.0065,0.4909,0.5026,호재(긍정)
29,20150330,"한국콜마, 中시장 공략 강화 올 매출 10% 늘것",◆ 기업분석 / 화장품 ODM업체 한국콜마 ◆ \n 한국콜마는 화장품 주문자상표부착...,161890,1,0.0004,0.0064,0.9932,호재(긍정)
40,20150514,"한국콜마, 화장품 사업 호조 지속될 전망 '목표가↑'-하이",[이데일리 박기주 기자] 하이투자증권이 14일 한국콜마(161890)에 대해 화장품...,161890,1,0.0011,0.0306,0.9683,호재(긍정)
44,20150514,"""한국콜마, 수출 확대로 할인 요인 해소 목표가↑""-KTB",[ 박희진 기자 ] KTB투자증권은 14일 한국콜마에 대해 올 1분기 실적에서 수출...,161890,1,0.0022,0.1758,0.8220,호재(긍정)
62,20150520,"한국콜마, 베이징법인 성장 구체화 '목표가↑'-교보",[이데일리 박기주 기자] 교보증권이 20일 한국콜마(161890)에 대해 베이징법인...,161890,1,0.0011,0.0314,0.9675,호재(긍정)
...,...,...,...,...,...,...,...,...,...
4128,20241106,"한국콜마, 화장품 출시 6개월까지 줄이는 서비스 구축",[대한경제=오진주 기자]한국콜마가 화장품 출시 기간을 줄이는 서비스를 구축했다. \...,161890,-1,0.6929,0.3034,0.0037,악재(부정)
4138,20241106,한국콜마 “화장품 개발 6개월 단축”,한국콜마가 “화장품 신제품 출시기간을 최대 6개월 단축하는 PPS를 본격 가동한다”...,161890,-1,0.5592,0.4368,0.0040,악재(부정)
4167,20241111,코스맥스 한국콜마 실적 랠리 'K뷰티 피크아웃' 우려 불식,화장품 제조업자개발생산(ODM) ‘투톱’인 코스맥스와 한국콜마가 올해 3분기 사상 ...,161890,1,0.0024,0.1081,0.8895,호재(긍정)
4172,20241112,"[단독] 한국콜마, '기술유출 사건' 6년 만 종지부..伊 인터코스 유죄 확정",서울의 한 대형마트에 화장품이 진열돼 있다. 뉴스1 \n[파이낸셜뉴스] 한국콜마의 ...,161890,-1,0.7164,0.2723,0.0114,악재(부정)


In [ ]:
df_hope.iloc[29,]

,29
일자,20150209
제목,"대웅제약-크리스탈지노믹스, 국산 신약 ‘아셀렉스캡슐’ 판권 계약"
본문,대웅제약은 바이오벤처 크리스탈지노믹스와 골관절염 치료 신약 아셀렉스캡슐(이하 아셀렉...
ticker,69620
pred_label,1
prob_neg,0.0048
prob_neu,0.0773
prob_pos,0.9179
sentiment,호재(긍정)


In [ ]:
df_hope.iloc[29,]['본문']

'대웅제약은 바이오벤처 크리스탈지노믹스와 골관절염 치료 신약 아셀렉스캡슐(이하 아셀렉스)의 국내 판권 계약을 체결했다. \n \n대웅제약은 이번 계약을 통해 국내에서 아셀렉스를 독점적으로 판매할 수 있는 권리를 얻게 된다.\xa0 \n \n골관절염 증상이나 징후를 완화해주는 아셀렉스는 국산 신약 22호로 국내외 시판중인 경구 골관절염 치료 약물 중 하루 복용량이 가장..'

In [ ]:
df_real = pd.read_csv('/content/drive/MyDrive/LG생활건강_predicted_results.csv')
df_real

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150102,LG생활건강 차석용 부회장 “해외사업 적극 전개로 시장 선도하자”,차석용 LG생활건강 부회장(사진제공=LG생활건강)차석용 LG생활건강 부회장은 2일 ...,51900,0,0.0874,0.9104,0.0022,중립
1,20150102,"LG생활건강, 4Q 실적 기대치 충족...목표가↑",[아시아경제 노태영 기자]KB투자증권은 2일 LG생활건강에 대해 4분기 실적이 시장...,51900,1,0.0001,0.0001,0.9999,호재(긍정)
2,20150102,"LG생활건강 “中 최우선 목표, 해외사업 적극전개”",[헤럴드경제=오연주 기자] LG생활건강은 올해 해외사업을 적극적으로 전개해 시장을 ...,51900,0,0.1438,0.8535,0.0027,중립
3,20150102,"LG생활건강, 4Q이어 내년에도 견조한 성장세 유지[KB투자證]",KB투자증권은 2일LG생활건강에 대해 4분기 영업이익성장률이 두 자릿수를 기록할 것...,51900,0,0.0164,0.9633,0.0203,중립
4,20150102,"[신년사]차석용 LG생활건강 부회장 ""해외 진출 새로운 기회 적극 추진할 것""",[차석용 LG생활건강 부회장] \n \n아주경제 한지연 기자 =차석용 LG생활건강 ...,51900,1,0.0002,0.0001,0.9998,호재(긍정)
...,...,...,...,...,...,...,...,...,...
6731,20241217,"LG생활건강, '다우존슨 지속가능경영지수' 7년 연속 편입",LG생활건강이 2024 다우존스 지속가능경영지수(Dow Jones Sustainab...,51900,0,0.0023,0.9964,0.0013,중립
6732,20241217,"LG생활건강, 대한민국 소비재 기업 최초 DJSI 월드 지수 7년 연속 편입",LG생활건강은 ‘2024 다우존스 지속가능경영지수(이하 DJSI)’ 평가에서 DJS...,51900,0,0.0023,0.9962,0.0015,중립
6733,20241218,"LG생활건강, 소비재 기업 최초 DJSI 월드 지수 7년 연속 편입",LG생활건강이 ‘2024 다우존스 지속가능경영지수(Dow Jones Sustaina...,51900,0,0.0018,0.9969,0.0013,중립
6734,20241220,"LG생활건강, 강아지 발씻자 '티슈형' 출시",LG생활건강은 강아지 발씻자의 ‘티슈형’ 제품을 고객들에게 선보인다고 20일 밝혔다...,51900,0,0.0026,0.9963,0.0011,중립


In [ ]:
df_real['sentiment'].value_counts()

,count
sentiment,
중립,5373
호재(긍정),946
악재(부정),417


In [ ]:
df_real_plus = pd.read_csv('/content/drive/MyDrive/LG생활건강_predicted_results_plus.csv')

In [ ]:
df_real_plus['sentiment'].value_counts()

,count
sentiment,
악재(부정),3595
호재(긍정),3141


In [ ]:
df_hope = pd.read_csv('/content/drive/MyDrive/뉴스분류/LG생활건강_predicted_results_hope.csv')

In [ ]:
df_hope['sentiment'].value_counts()

,count
sentiment,
중립,5045
호재(긍정),1087
악재(부정),604


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/news_labeling_100종목/LG생활건강_clean_labeling.csv')

In [ ]:
df['label'].value_counts()

,count
label,
0,5409
1,929
-1,398


In [ ]:
sum(df['제목'] == df_hope['제목'])

6736

In [ ]:
idx_list = []
for idx, sentiment in enumerate(df['label']):
  if sentiment != df_hope['pred_label'][idx]:
    idx_list.append(idx)

KeyError: 'label'

In [ ]:
df.iloc[115]

,115
일자,20150225
제목,"LG생활건강, 잘 나가는 화장품에 非화장품 부문 가세 목표가 '상향'"
본문,[아시아경제 임철영 기자]NH투자증권은 25일 LG생활건강에 대해 화장품 이익 기여...
ticker,51900
label,0


In [ ]:
df_hope.iloc[115]

,115
일자,20150225
제목,"LG생활건강, 잘 나가는 화장품에 非화장품 부문 가세 목표가 '상향'"
본문,[아시아경제 임철영 기자]NH투자증권은 25일 LG생활건강에 대해 화장품 이익 기여...
ticker,51900
pred_label,1
prob_neg,0.0063
prob_neu,0.2003
prob_pos,0.7934
sentiment,호재(긍정)


In [ ]:
df_hope.iloc[115]['본문']

'[아시아경제 임철영 기자]NH투자증권은 25일 LG생활건강에 대해 화장품 이익 기여도가 빠르게 높아지고 있고, 할인요인 이었던 생활용품과 음료 턴어라운드 가능성이 있다며 목표주가를 기존 82만원에서 89만원으로 7만원 상향조정했다. 투자의견은 \'매수\'를 유지했다.한국희 애널리스트는 "중국내 온라인 구매 급증에 대한 유연한 대응으로 중국발 수요를 흡수해 ..'

In [ ]:
df.iloc[1070]

,1070
일자,20160727
제목,"LG생활건강, 2 4분기 실적 호조에도 하락세"
본문,LG생활건강(051900)이 올 2 4분기 실적 호조에도 불구하고 차익 실현 매물에...
ticker,51900
label,0


In [ ]:
df_hope.iloc[1070]['본문']

'LG생활건강(051900)이 올 2 4분기 실적 호조에도 불구하고 차익 실현 매물에 하락세다. \n \n27일 오전 9시31분 현재 유가증권시장에서 전날보다 6.32%(6만9,000원) 내린 102만3,000원에 거래되고 있다. \n \nLG생활건강은 연결기준 2 4분기 영업이익이 2,254억원으로 작년 동기보다 34.1% 늘었다고 전날 밝혔다. 그간 고성장 ..'

In [ ]:
df.iloc[6325]

,6325
일자,20240617
제목,"LG생활건강, 중국 실적 회복 기대 목표가↑-NH"
본문,[이데일리 이정현 기자] NH투자증권은 LG생활건강(051900)에 대해 “중국 이...
ticker,51900
label,0


In [ ]:
df_hope.iloc[6325]['본문']

'[이데일리 이정현 기자] NH투자증권은 LG생활건강(051900)에 대해 “중국 이커머스 채널에서 양호한 성과를 보이고 있으며, 면세 역시 전분기 대비 증가하며 손익 안정화가 될 것”이라 진단했다. 투자의견은 ‘매수’, 목표가는 50만원으로 ‘상향’했다.\n\n정지윤 NH투자증권 연구원은 17일 보고서에서 “LG생활건강의 2분기 영업이익은 컨센서스에 부합할..'

In [ ]:
df.iloc[6675]

,6675
일자,20241129
제목,"'일본 시장 공략 성공' LG생활건강, 11월 행사 매출 급증"
본문,[서울=뉴스핌] 조민교 기자 = LG생활건강은 일본 이커머스 플랫폼 큐텐(Qoo10...
ticker,51900
label,0


In [ ]:
df_hope.iloc[6675]['본문']

"[서울=뉴스핌] 조민교 기자 =\xa0LG생활건강은 일본 이커머스 플랫폼 큐텐(Qoo10)의 11월 '메가와리' 행사에서 전년 대비 약 4배에 달하는 매출 성장을 달성했다고 29일 전했다. 일본 시장에서 LG생활건강의 브랜드와 제품력이 인정받았다는 평가다. \n \n\n 이번 행사는 11월 15일부터 27일까지 진행됐으며 LG생활건강의 매출액은 지난해 대비 292.."

In [ ]:
df.iloc[6400]

,6400
일자,20240725
제목,"LG생활건강, 2분기 영업익 1585억 전년比 0.4%↑"
본문,"LG생활건강은 2분기 매출 1조7597억원, 영업이익 1585억원을 기록했다고 25..."
ticker,51900
label,0


In [ ]:
df_hope.iloc[6400]['본문']

'LG생활건강은 2분기 매출 1조7597억원, 영업이익 1585억원을 기록했다고 25일 공시를 통해 밝혔다.\n\n\n\n지난해와 비교해 2분기 전사 매출은 2.7% 감소했고, 영업이익은 0.4% 증가했다.\n\n\n\n사업별로는 화장품, 생활용품, 음료 모두 매출은 감소했으나 화장품과 생활용품 부문의 영업이익은 증가했다.\n\n\n\n2분기 화장품 사업 매출은 지난해 같은기..'

In [ ]:
df.iloc[616]

,616
일자,20151208
제목,"LG생활건강, 수익성 개선 기대 목표가↑-HMC투자증권"
본문,"HMC투자증권은 8일 LG생활건강에 대해 프레스티지 화장품 라인업 확보, 생활용품 ..."
ticker,51900
label,0


In [ ]:
df_hope.iloc[616]['본문']

'HMC투자증권은 8일 LG생활건강에 대해 프레스티지 화장품 라인업 확보, 생활용품 수요 확대 등이 향후 매출 성장과 수익성 개선을 주도할 것으로 내다보고, 목표주가를 종전 110만원에서 117만원으로 올렸다. 내년 LG생활건강의 매출액과 영업이익은 6조2492억원과 8234억원으로 각각 올해보다 16.4%, 19.6% 증가할 것으로 전망했다. \n \n 조..'

In [ ]:
for idx in idx_list: